In [ ]:
from pyspark.sql import SparkSession

# Set your own jar path
jar_path = "path/to/postgresql-42.x.x.jar"

# SparkSession Start
spark = SparkSession.builder \
    .appName("TestSession") \
    .master("local[*]") \
    .config("spark.jars", jar_path) \
    .getOrCreate()

print(spark.version)

3.5.2


In [ ]:
# Set your data base properties
jdbc_url = "jdbc:postgresql://<host>:<port>/<database>"
db_properties = {
    "user": "<username>",
    "password": "<password>",
    "driver": "org.postgresql.Driver"
}

# Read the data to the database
df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "shopping_trends") \
    .option("user", db_properties["user"]) \
    .option("password", db_properties["password"]) \
    .option("driver", db_properties["driver"]) \
    .load()

# Show data
df.show()

+---+---+----------+-----------+------+-------------+----+---------+------+------+
| id|age|      item|   category|amount|     location|size|    color|season|rating|
+---+---+----------+-----------+------+-------------+----+---------+------+------+
|  1| 55|    Blouse|   Clothing|    53|     Kentucky|   L|     Gray|Winter|   3.1|
|  2| 19|   Sweater|   Clothing|    64|        Maine|   L|   Maroon|Winter|   3.1|
|  3| 50|     Jeans|   Clothing|    73|Massachusetts|   S|   Maroon|Spring|   3.1|
|  4| 21|   Sandals|   Footwear|    90| Rhode Island|   M|   Maroon|Spring|   3.5|
|  5| 45|    Blouse|   Clothing|    49|       Oregon|   M|Turquoise|Spring|   2.7|
|  6| 46|  Sneakers|   Footwear|    20|      Wyoming|   M|    White|Summer|   2.9|
|  7| 63|     Shirt|   Clothing|    85|      Montana|   M|     Gray|  Fall|   3.2|
|  8| 27|    Shorts|   Clothing|    34|    Louisiana|   L| Charcoal|Winter|   3.2|
|  9| 26|      Coat|  Outerwear|    97|West Virginia|   L|   Silver|Summer|   2.6|
| 10

In [ ]:
#   Register the DataFrame as a SQL temporary view in order to be able to use SQL queries
df.createOrReplaceTempView("data_table")
 

result = spark.sql("SELECT item, SUM(amount) AS Total_Spending \
                    FROM data_table \
                    GROUP BY item \
                    ORDER BY Total_Spending DESC;")
 

result.show()

+----------+--------------+
|      item|Total_Spending|
+----------+--------------+
|    Blouse|         10410|
|     Shirt|         10332|
|     Dress|         10320|
|     Pants|         10090|
|   Jewelry|         10010|
|Sunglasses|          9649|
|      Belt|          9635|
|     Scarf|          9561|
|   Sweater|          9462|
|    Shorts|          9433|
|     Skirt|          9402|
|       Hat|          9375|
|      Coat|          9275|
|     Socks|          9252|
|    Jacket|          9249|
|   T-shirt|          9248|
|     Shoes|          9240|
|   Sandals|          9200|
|     Boots|          9018|
|   Handbag|          8857|
+----------+--------------+
only showing top 20 rows



In [ ]:
result = spark.sql("SELECT category, AVG(`rating`) AS Average_Rating \
                    FROM data_table \
                    GROUP BY category \
                    ORDER BY Average_Rating DESC;")
 

result.show()

+-----------+------------------+
|   category|    Average_Rating|
+-----------+------------------+
|   Footwear|3.7906510851419055|
|Accessories|3.7686290322580676|
|  Outerwear| 3.746913580246914|
|   Clothing|3.7231433506044884|
+-----------+------------------+



In [15]:
age_group_df = spark.sql("SELECT CASE \
           WHEN `age` < 21 THEN 'Under 21' \
           WHEN `age` BETWEEN 21 AND 30 THEN '21-30' \
           WHEN `age` BETWEEN 31 AND 40 THEN '31-40' \
           WHEN `age` BETWEEN 41 AND 50 THEN '41-50' \
           WHEN `age` > 50 THEN 'Above 50' \
       END AS `Age_Group`, \
       AVG(`amount`) AS `Avg_Purchase_Amount` \
FROM data_table \
GROUP BY `Age_Group` \
ORDER BY `Avg_Purchase_Amount` DESC;")
 

age_group_df.show()

+---------+-------------------+
|Age_Group|Avg_Purchase_Amount|
+---------+-------------------+
|    21-30|  60.75305291723202|
| Above 50|  59.94579945799458|
|    31-40|  59.57655172413793|
| Under 21|   58.9811320754717|
|    41-50|  58.83866666666667|
+---------+-------------------+



In [14]:
result = spark.sql("SELECT id, \
            SUM(amount) AS `Total_Spending`,  \
            AVG(rating) AS `Avg_Review_Rating` \
            FROM data_table \
            GROUP BY `id` \
            ORDER BY `Total_Spending` DESC, `Avg_Review_Rating` DESC \
            LIMIT 5;")
 

result.show()

+----+--------------+-----------------+
|  id|Total_Spending|Avg_Review_Rating|
+----+--------------+-----------------+
|2072|           100|              4.8|
| 249|           100|              4.8|
|3838|           100|              4.8|
|2286|           100|              4.6|
|1301|           100|              4.5|
+----+--------------+-----------------+



In [19]:
spark.stop()